In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# import os


In [2]:
# file_path = "/content/gdrive/MyDrive/Colab Notebooks/deplying_models/diabetic_detection"

In [3]:
# os.chdir(file_path)

In [4]:
# pip install scikit-learn==0.21.3

In [5]:
import pickle
import pandas as pd


In [6]:
def model_predict(NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age):
    # file paths/name
    filename_model = 'svc_model_diabetic_detection.sav'
    filename_scaler = 'minmax_scaler_diabetic_detection.sav'

    #  load the scaler and model from disk
    loaded_scaler = pickle.load(open(filename_scaler, 'rb'))
    loaded_model = pickle.load(open(filename_model, 'rb'))

    # params to array
    df = [[NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age]]
    print(df)
    # scale
    new_df_scaled = loaded_scaler.transform(df)
    print(new_df_scaled)
    # predict
    result = loaded_model.predict(new_df_scaled)[0]

    return result

In [7]:
model_predict(6, 168, 72, 35, 0, 43.6, 0.627, 65)

[[6, 168, 72, 35, 0, 43.6, 0.627, 65]]
[[0.35294118 0.84422111 0.59016393 0.55555556 0.         0.64977645
  0.23441503 0.73333333]]


1

In [8]:
# Sample Query Parameters
# "NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age", "HasDiabetes"
q_params = f"?NumTimesPrg={6}&PlGlcConc={168}&BloodP={72}&SkinThick={35}&TwoHourSerIns={0}&BMI={43.6}&DiPedFunc={0.627}&Age={65}"

In [9]:
q_params

'?NumTimesPrg=6&PlGlcConc=168&BloodP=72&SkinThick=35&TwoHourSerIns=0&BMI=43.6&DiPedFunc=0.627&Age=65'

In [10]:
# # fastapi and uvicorn - creating api and server
# !pip install fastapi uvicorn

# # ngrok - providing public urls for testing our api
# !pip install nest-asyncio pyngrok 

In [11]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# file paths/name
filename_model = 'svc_model_diabetic_detection.sav'
filename_scaler = 'minmax_scaler_diabetic_detection.sav'

#  load the scaler and model from disk
loaded_scaler = pickle.load(open(filename_scaler, 'rb'))
loaded_model = pickle.load(open(filename_model, 'rb'))


@app.get('/params/')
def mode_predict(NumTimesPrg : int, PlGlcConc : int, BloodP : int, SkinThick : int, TwoHourSerIns : int, BMI : float, DiPedFunc : float, Age : int):

    # params to array
    df = [[NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age]]
   
    # scale
    new_df_scaled = loaded_scaler.transform(df)
    
    # predict
    result = loaded_model.predict(new_df_scaled)[0]

    return {"result": str(result)}
    # return NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age

In [12]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

url = ngrok.connect(port=80)
print('Public URL:', url)
print('Sample URL w/ input:', url.data['public_url'] + "/params/" + q_params)
nest_asyncio.apply()

uvicorn.run(app, port=80)

INFO:     Started server process [48870]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 13] error while attempting to bind on address ('127.0.0.1', 80): permission denied
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Public URL: NgrokTunnel: "http://361390c3a4b7.ngrok.io" -> "http://localhost:80"
Sample URL w/ input: http://361390c3a4b7.ngrok.io/params/?NumTimesPrg=6&PlGlcConc=168&BloodP=72&SkinThick=35&TwoHourSerIns=0&BMI=43.6&DiPedFunc=0.627&Age=65


SystemExit: 1

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

In [ ]:
ngrok.get_tunnels()

t=2021-01-22T05:50:44+0800 lvl=warn msg="failed to open private leg" id=03f5e13c5405 privaddr=localhost:80 err="dial tcp [::1]:80: connect: connection refused"
t=2021-01-22T05:50:45+0800 lvl=warn msg="failed to open private leg" id=4282804a5fd7 privaddr=localhost:80 err="dial tcp [::1]:80: connect: connection refused"
